# Poro quantized LLM model By TheBloke implemented on VertexAI GCP 

In [54]:
# Model: https://huggingface.co/TheBloke/Poro-34B-GPTQ

In [ ]:
# GCP instance
# Debian 11 + Python 3 (with Intel MKL and CUDA 11.8)
# 2 x L4 NVIDIA GPUs
# 24 vCPUs, 96 GB RAM

In [55]:
# environment: install requirements.txt

## Import libraries

In [1]:
import torch

In [2]:
torch.__version__

'2.2.0+cu121'

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.zeros(1).cuda()

tensor([0.], device='cuda:0')

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [39]:
from transformers import pipeline

### Quantized model using GPUs

In [9]:
model_name_or_path = "TheBloke/Poro-34B-GPTQ"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

tokenizer_config.json: 100%|██████████| 286/286 [00:00<00:00, 172kB/s]
tokenizer.json: 100%|██████████| 5.64M/5.64M [00:00<00:00, 8.21MB/s]
special_tokens_map.json: 100%|██████████| 545/545 [00:00<00:00, 334kB/s]


In [10]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="gptq-4bit-32g-actorder_True")

config.json: 100%|██████████| 1.00k/1.00k [00:00<00:00, 249kB/s]
model.safetensors: 100%|██████████| 22.9G/22.9G [24:24<00:00, 15.7MB/s]
generation_config.json: 100%|██████████| 132/132 [00:00<00:00, 80.0kB/s]


### Prompt and prompt template (in English)

In [44]:
prompt = "It was a dark and stormy night"

In [45]:
prompt_template=f'''{prompt}
'''

### Input

In [46]:
input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()

### Output using model.generate

In [49]:
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.9, top_k=50, max_new_tokens=512)

In [50]:
print(tokenizer.decode(output[0]))

It was a dark and stormy night
I was a young man, on my first night shift in the ER. It was just me, the resident and the charge nurse. We were all alone in the ER, and it was very quiet. All of a sudden, I heard a loud bang, and the lights went out. I looked up to see what had caused the bang, and I saw a huge shadowy figure standing in the middle of the room. I was terrified, but I had to do my job, so I went to investigate. As I got closer to the figure, I could see that it was a giant black dog with glowing red eyes. I screamed and ran back to the nurse's station, but the dog was already on me. I tried to fight it off, but it was too strong. It bit me hard, and I fell to the floor. I screamed again, and the nurse came running. She saw the dog and screamed, too. She grabbed me and pulled me to safety. As we were running, I could hear the dog growling and barking at us. We made it to the nurse's station, and the nurse called the police. The police came quickly, and they shot the dog 

### Ourput using transformers' pipeline

In [51]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.5,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.1
)

In [52]:
print(pipe(prompt_template)[0]['generated_text'])

It was a dark and stormy night
I went to the window, I looked out on the rain,
And what should I see?
But only darkness.
The next morning when I woke up it had stopped raining. The clouds were gone from over my head!
How do you know that God is real? How can we prove his existence if he doesn't exist in this world of matter like us humans are made of (or at least that's how science says)?
Well…the Bible has been proven by scientists time after time! It contains information about scientific events which happened thousands or millions years before they actually did happen!! And then there have also been other things discovered through research such as fossils found under water with dates written upon them showing these objects lived hundreds of thousands/millions ago!!
There have even been some amazing archaeological discoveries lately where archaeologists find items left behind for centuries buried deep underground – yet still perfectly preserved today!!! Archaeologist dig into the grou

### Prompt in Finnish

In [34]:
prompt = "Oli synkkä ja myrskyinen yö"

In [35]:
prompt_template=f'''{prompt}
'''

In [36]:
input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()

### Output with model.generate

In [37]:
output = model.generate(inputs=input_ids, temperature=0.5, do_sample=True, top_p=0.9, top_k=50, max_new_tokens=512)

In [38]:
print(tokenizer.decode(output[0]))

Oli synkkä ja myrskyinen yö
Kun heräsin, oli jo pimeää. En nähnyt mitään, mutta tunsin, että jotain oli vialla. Oli kylmä ja kostea, ja olin aivan yksin. Olin kuullut yöllä outoja ääniä, mutta en ollut uskaltanut mennä katsomaan. Nyt olin varma, että jokin oli pielessä. Menin varovasti ulos ja näin, että koko kaupunki oli pimeänä. Kaikkialla oli hiljaista. En nähnyt ketään, mutta kuulin, kuinka joku juoksi minua kohti. Juoksin nopeasti takaisin sisälle. En uskaltanut mennä ulos enää sinä yönä.</s>


### Output using pipeline

In [42]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.5,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.1
)

In [43]:
print(pipe(prompt_template)[0]['generated_text'])

Oli synkkä ja myrskyinen yö
kun saavuin kotiin.
Ei ollut ketään kotona,
ei edes kissaa näkynyt mailla halmeilla!
Mitä ihmettä?
Mistä on kyse?!
Kuka täällä oikein asustaa...?
Missä kaikki ovat??
|Kaikkialla oli vain tyhjyyttä.|
Lopulta löysin sen...
Se istui sängyllä, tuijotti minua suoraan silmiin.
|"Hei! Minä olen kissa. Kuka sinä olet?|
En ole nähnyt sinua koskaan aikaisemmin..."
|"Mikä sinun nimesi on? En muista sitä enää..."|
|"Minä olen Kissa. Mitä sinä teet täällä?"|
|"En tiedä. Olen eksynyt tänne joskus kauan sitten.|
|"Onko sinulla nälkä? Minulla ainakin olisi.."|
|"No ei minullakaan kyllä kovin hyvin mene...."|
|"Kyllä tämä tästä vielä iloksi muuttuu..."|
|"Ai niin! Minun nimeni onkin Pulla! "|
|"Selvä juttu. Hei hei! Nähdään taas!"|
|Ja näin päättyi tarina Kissan ja Pullan kohtaamisesta.|
Ei kommentteja:
Lähetä kommentti
